In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import pandas as pd
from tqdm import tqdm

from car_detection import RAW_DATA_DIR, PROCESSED_DATA_DIR
from car_detection.box_utils import scale_bbox, bbox2yolo, save_bboxes
from car_detection.image_utils import resize_with_pad

In [ ]:
# Raw data paths
images_dname = "images"
labels_fname = "labels.csv"

raw_images_dpath = RAW_DATA_DIR / images_dname
raw_labels_fpath = RAW_DATA_DIR / labels_fname

In [ ]:
# Processed data paths
processed_labels_dname = "labels"

processed_imgs_dpath = PROCESSED_DATA_DIR / images_dname
processed_labels_dpath = PROCESSED_DATA_DIR / processed_labels_dname

processed_imgs_dpath.mkdir(exist_ok=True)
processed_labels_dpath.mkdir(exist_ok=True)

In [ ]:
lst_images_dir = list(raw_images_dpath.rglob("*.[jpg][png][jpeg]"))
len(lst_images_dir), lst_images_dir[:3]

In [ ]:
labels_df = pd.read_csv(raw_labels_fpath)
labels_df.head()

In [ ]:
img_names, bboxes = labels_df.iloc[:, 0].to_numpy(), labels_df.iloc[:, 1:].to_numpy()
img_names[:3], bboxes[:3]

### Scale bboxes to 640x640 && format to normalized xywh

In [ ]:
orig_imgsz = (380, 676)
target_imgsz = (640, 640)

resized_bboxes = [scale_bbox(bbox, orig_imgsz, target_imgsz) for bbox in bboxes]
xywh_norm_bboxes = [bbox2yolo(bbox, target_imgsz) for bbox in resized_bboxes]

In [ ]:
# Write bboxes to txt files
marked_imgs_dpath = set()
for img_name, bbox in tqdm(zip(img_names, xywh_norm_bboxes), desc="Labels", total=len(img_names)):
    img_fpath = raw_images_dpath / img_name
    marked_imgs_dpath.add(img_fpath)
    
    label_fname = f"{img_fpath.stem}.txt"
    label_fpath = processed_labels_dpath / label_fname
    
    save_bboxes(label_fpath, [bbox], [0])

### Resize images (with markup only) with aspect ratio And save

In [ ]:
# Images with markup
len(marked_imgs_dpath)

In [ ]:
for img_fpath in tqdm(marked_imgs_dpath):
    img = cv2.imread(img_fpath)
    resized_img = resize_with_pad(img, target_imgsz)

    resized_img_fpath = processed_imgs_dpath / img_fpath.name
    cv2.imwrite(resized_img_fpath, resized_img)